In [1]:
%load_ext autoreload
%autoreload 2
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)


In [2]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, material = read_VSP_group(pappas_only=False)

D = count_matrix(groups_VSP, raw_feature.shape[0])
clusters_VSP = spectralClusteringDBSCAN(D, threshold=0,eps=0.001)

print(raw_feature.shape)
print(label_material.shape)

INFO:visiprog.data:After removing non zero entries, N is 3708
INFO:visiprog.data:Number of disconnected elements is 161

(5245, 82)
(5245,)


In [3]:
# print(clusters_VSP)

In [3]:
label_visiprog = get_VSP_label(groups_VSP, raw_feature.shape[0])
label_VSP_clusters = get_VSP_label(clusters_VSP, raw_feature.shape[0])

In [4]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)

In [14]:
accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp_original = []
accuracy_vsp = []

list_img = read_img_list()

n_cluster = 1

for i in range(10):
    X_train, X_test, X_STSIMM_train, X_STSIMM_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
    train_test_split(raw_feature, X_STSIMM, label_material, label_VSP_clusters, \
                     test_size=0.2, stratify=label_material)
    
    res = kmean_subclass(X_STSIMM_train, Y_mat_train, X_STSIMM_test, Y_mat_test, n_cluster)
    accuracy_STSIMM.append(res['accuracy'])
    
    model = LFDA()
    try:
        model.fit(X_train, Y_mat_train)
        X_mat_train = model.transform(X_train)
        X_mat_test = model.transform(X_test)
        res = kmean_subclass(X_mat_train, Y_mat_train, X_mat_test, Y_mat_test, n_cluster)
        accuracy_mat.append(res['accuracy'])
    except Exception as e:
        acuracy_mat.append(np.nan)
    
    model_vsp_original = LFDA()
    try:

        idx_outlier = np.where(Y_VSP_train == 0)[0]
        X_train_without_outlier = X_train[~idx_outlier,:]
        Y_VSP_train_without_outlier = Y_VSP_train[~idx_outlier]
        model_vsp_original.fit(X_train_without_outlier, Y_VSP_train_without_outlier)
        
        X_vsp_original_train = model_vsp_original.transform(X_train)
        X_vsp_original_test = model_vsp_original.transform(X_test)
        res_vsp_original = kmean_subclass(X_vsp_original_train, Y_mat_train, \
                                          X_vsp_original_test, Y_mat_test, n_cluster)
        print(res_vsp_original['accuracy'])
        accuracy_vsp_original.append(res_vsp_original['accuracy'])
    except Exception as e:
        print(e)
        accuracy_vsp_original.append(np.nan)


    model_vsp = LFDA_VISIPROG()
    try:
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_test = model_vsp.transform(X_test)
        
        # remove trained data which are not labeled by VSP
        indx = np.where(Y_VSP_train != 0)[0]
        X_vsp_train = model_vsp.transform(X_train[indx])
        res_vsp = kmean_subclass(X_vsp_train, Y_mat_train[indx], X_vsp_test, Y_mat_test, n_cluster)
        accuracy_vsp.append(res_vsp['accuracy'])
    except Exception as e:
        accuracy_vsp.append(np.nan)
        

df = pd.DataFrame({'STSIM-M':accuracy_STSIMM,'visiprog':accuracy_vsp, \
                   'visiprog_original': accuracy_vsp_original, 'material':accuracy_mat}, \
                  columns=['STSIM-M','visiprog_original','visiprog','material'])
df.loc['average'] = df.mean()
display(df)


df.to_csv('thrasos/within_fromcluster_k{}.csv'.format(n_cluster))



INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[10 46 52 ..., 60 55 17]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[10 46 52 ..., 60 55 17]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[10 46 52 ..., 60 55 17]
INFO:visiprog.evaluate:Running on test set
0.733079122974
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[46 52 50 ..., 30 41 17]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[38 40 12 ..., 23 23  6]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[38 40 12 ..., 23 23  6]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[38 40 12 ..., 23 23  6]
INFO:visiprog.evaluate:Running on test set
0.76167778837
INFO:visiprog.evaluate:KMeans clustering
INFO:v

,STSIM-M,visiprog_original,visiprog,material
0,0.817922,0.733079,0.766444,0.826501
1,0.837941,0.761678,0.798856,0.852240
2,0.816969,0.755005,0.795996,0.849380
3,0.818875,0.775977,0.781697,0.845567
4,0.820782,0.794090,0.775024,0.837941
5,0.835081,0.735939,0.772164,0.848427
6,0.824595,0.762631,0.785510,0.857007
7,0.838894,0.741659,0.803622,0.879886
8,0.843661,0.740705,0.789323,0.857007
9,0.826501,0.788370,0.776930,0.853194
